test hyperparameter tuned RFC and XGB

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install scikit-optimize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 3.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import joblib
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from scipy.stats import randint, uniform
# TF IDF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split # fnding max_features
# from sklearn.ensemble import
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
# import joblib
import os

# hyperparameter tuning
 # to look availiable hyperparameters
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV #

from hyperopt import fmin, tpe, hp

import scipy.stats as stats
import xgboost as xgb

from sklearn.model_selection import GridSearchCV

import spacy
import ast

# models for hp tuning
from sklearn.ensemble import  RandomForestClassifier # hp tunned
from sklearn.tree import DecisionTreeClassifier # hp tuned
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier # hp tunned
from sklearn.neighbors import KNeighborsClassifier #hp tuned
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC

## tuning parameters for sampling techniques
## under sampling
from imblearn.under_sampling import  RandomUnderSampler, EditedNearestNeighbours, CondensedNearestNeighbour, TomekLinks, ClusterCentroids, RepeatedEditedNearestNeighbours, NearMiss, NeighbourhoodCleaningRule, OneSidedSelection, AllKNN, InstanceHardnessThreshold

## over sampling
from imblearn.over_sampling import SMOTE, ADASYN, BorderlineSMOTE, SVMSMOTE, KMeansSMOTE, RandomOverSampler, SMOTENC

## over and under sampling
from imblearn.combine import SMOTEENN, SMOTETomek


from imblearn.pipeline import Pipeline

from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer


In [ ]:
!pip install lightgbm

In [ ]:
!pip uninstall -y scikit-learn
!pip install scikit-learn==1.5.2

Found existing installation: scikit-learn 1.6.1
Uninstalling scikit-learn-1.6.1:
  Successfully uninstalled scikit-learn-1.6.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 86.7 MB/s eta 0:00:00


In [ ]:
dev_df_train = pd.read_csv('/content/drive/MyDrive/Thesis/data/dev/second_dev_train.csv')
dev_df_test = pd.read_csv('/content/drive/MyDrive/Thesis/data/dev/first_dev_test.csv')
df_test_org =  pd.read_csv('/content/drive/MyDrive/Thesis/data/train/new_11.csv')
df_test = pd.read_csv('/content/drive/MyDrive/Thesis/data/test/english_test_11_2.csv')
gold_test = pd.read_csv('/content/drive/MyDrive/Thesis/data/test/CT24_checkworthy_english_test_gold.tsv',delimiter='\t')

In [ ]:
dev_df_test.columns

Index(['Sentence_id', 'Text', 'class_label', 'label', 'Tokens', 'text_length',
       'cleaned_text', 'cleaned_text_length', 'roberta_sentiment',
       'count_tokens', 'counts', 'roberta_sent_pos', 'roberta_sent_neg',
       'roberta_sent_mixed', 'roberta_sentiment_class', 'names',
       'organizations', 'dates', 'punctuation_count', 'embedding',
       'converted_embedding', 'flattened_embedding'],
      dtype='object')

In [ ]:
print(dev_df_train['class_label'].value_counts())
print(dev_df_test['class_label'].value_counts())
print(df_test_org['class_label'].value_counts())
# print(df_test['class_label'].value_counts())
print(gold_test['class_label'].value_counts())

class_label
No     794
Yes    238
Name: count, dtype: int64
class_label
No     210
Yes    108
Name: count, dtype: int64
class_label
No     17088
Yes     5413
Name: count, dtype: int64
class_label
No     253
Yes     88
Name: count, dtype: int64


In [ ]:
pca_path = '/content/drive/MyDrive/Thesis/models/pca_356.pkl'
pca = joblib.load(pca_path)

# tf idf
vectorizer_path = '/content/drive/MyDrive/Thesis/models/tfidf_vectorizer_4300.pkl'
# tfidf_vect = TfidfVectorizer(lowercase = True, analyzer='word', stop_words = 'english', ngram_range=(1, 2))
tfidf_vect = joblib.load(vectorizer_path)

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator PCA from version 1.5.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 1.5.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator TfidfVectorizer from version 1.5.2 when using version 1.6.1. This might lead to breaking code or invalid 

In [ ]:
dev_df_train['class_label'].value_counts()

,count
class_label,
No,794
Yes,238


In [ ]:
df_test['Text'].iloc[0]

'They said they were just going to get inspectors in.'

In [ ]:
dev_df_train.head()

,Unnamed: 0,Sentence_id,Text,class_label,Tokens,text_length,cleaned_text,cleaned_text_length,roberta_sentiment,roberta_sent_pos,...,names,organizations,dates,punctuation_count,embedding,converted_embedding,flattened_embedding,label,joined_tokens,text-tfidf
0,0,26,"You know, I saw a movie - ""Crocodile Dundee.""",No,"['know', 'saw', 'movie', 'Crocodile', 'Dundee']",45,know saw movie crocodile dundee,31,"{'positive_score': 0.2028631716966629, 'negati...",0.202863,...,1,0,0,0,[[ 4.17878002e-01 -5.71067393e-01 3.49187367e...,[[ 4.17878002e-01 -5.71067393e-01 3.49187367e...,[ 4.17878002e-01 -5.71067393e-01 3.49187367e-...,0,know saw movie Crocodile Dundee,know see movie Crocodile Dundee
1,1,80,We're consuming 50 percent of the world's coca...,Yes,"['consuming', '50', 'percent', 'worlds', 'coca...",50,consuming 50 percent worlds cocaine,35,"{'positive_score': 0.014054246246814728, 'nega...",0.014054,...,0,0,0,0,[[-8.71186852e-02 3.84377271e-01 -1.50025383e...,[[-8.71186852e-02 3.84377271e-01 -1.50025383e...,[-8.71186852e-02 3.84377271e-01 -1.50025383e-...,1,consuming 50 percent worlds cocaine,consume 50 percent world cocaine
2,2,129,That answer was about as clear as Boston harbor.,No,"['answer', 'clear', 'Boston', 'harbor']",48,answer clear boston harbor,26,"{'positive_score': 0.10697311162948608, 'negat...",0.106973,...,0,0,0,0,[[-2.70109117e-01 -4.56845388e-03 -1.14797592e...,[[-2.70109117e-01 -4.56845388e-03 -1.14797592e...,[-2.70109117e-01 -4.56845388e-03 -1.14797592e-...,0,answer clear Boston harbor,answer clear Boston harbor
3,3,131,Let me help the governor.,No,"['Let', 'help', 'governor']",25,let help governor,17,"{'positive_score': 0.2973315715789795, 'negati...",0.297332,...,0,0,0,0,[[ 2.28126682e-02 1.58043981e-01 1.33417547e...,[[ 2.28126682e-02 1.58043981e-01 1.33417547e...,[ 2.28126682e-02 1.58043981e-01 1.33417547e-...,0,Let help governor,let help governor
4,4,172,We've run up more debt in the last eight years...,Yes,"['Weve', 'run', 'debt', 'last', 'eight', 'year...",125,weve run debt last eight years presidents geor...,81,"{'positive_score': 0.014272903092205524, 'nega...",0.014273,...,2,0,1,0,[[ 3.12675953e-01 4.28073525e-01 -1.17022857e...,[[ 3.12675953e-01 4.28073525e-01 -1.17022857e...,[ 3.12675953e-01 4.28073525e-01 -1.17022857e-...,1,Weve run debt last eight years presidents Geor...,we ve run debt last eight year president Georg...


In [ ]:
# convert columns to the right formation
# train
# test
# tfidf -> 50000

Index(['Sentence_id', 'Text', 'class_label'], dtype='object')

In [ ]:
# dev_df_train['text-tfidf']

In [ ]:
# new_df.head()
# new_df.rename(columns={'converted_embedding': 'embedding'}, inplace=True)

In [ ]:
def joined_tokens(tokens):
    token_list = ast.literal_eval(tokens)
    return ' '.join(token_list)

In [ ]:
nlp = spacy.load('en_core_web_sm')
def lemmatiz_text(text):
    doc = nlp(text)
    lemma_tokens = [token.lemma_ for token in doc]
    lemma_text = ' '.join(lemma_tokens)
    return lemma_text

In [ ]:
# join the tokens
dev_df_train['joined_tokens'] = dev_df_train['Tokens'].apply(joined_tokens)

# lemmatize
dev_df_train['text-tfidf'] = dev_df_train['joined_tokens'].apply(lemmatiz_text)

In [ ]:
dev_df_train =  dev_df_train.dropna()

In [ ]:
dev_df_train.shape

(1026, 25)

In [ ]:
# new_df.sample(2)

In [ ]:
# # tf idf
# text = dev_df_train['text-tfidf']
# vectorizer_path = '/content/drive/MyDrive/Thesis/models/tfidf_vectorizer_4300.pkl'
# # tfidf_vect = TfidfVectorizer(lowercase = True, analyzer='word', stop_words = 'english', ngram_range=(1, 2))
# tfidf_vect = joblib.load(vectorizer_path)
dev_text = dev_df_train['text-tfidf']
dev_tfidf_features = tfidf_vect.transform(dev_text)

In [ ]:
dev_tfidf_features.shape

(1026, 4300)

In [ ]:
# best_max_features = 10000 #  from ipynib checkwprthy_3

In [ ]:
# max_feature_range
# text_data = new_df['Text']
# labels = new_df['label']
# tfidf_vect = TfidfVectorizer(lowercase = True, analyzer='word', stop_words = 'english', ngram_range=(1, 2), max_features = best_max_features) # 1000
# tfidf_features = tfidf_vect.fit_transform(text_data)

In [ ]:
def convert_to_numpy_array(embd_str):
    #print(f"string: {embd_str}")
    #print(type(embd_str))
    embd_str_clean = embd_str.replace("\n", " ").replace(" ", " ")
    #print(type(embd_str_clean))
    #print(f"string cleansing: {embd_str_clean}")
    embd_str_clean = embd_str_clean.strip('[]')
    #print(type(embd_str_clean))
    #print(f"removing the outer brackets {embd_str_clean}")
    embd_list = embd_str_clean.split()
    #print(f"Split the string into individual values {embd_list}")
    embed_array = np.array(embd_list, dtype = float)
    #print(f"list to array {embed_array}")
    return embed_array

In [ ]:
dev_additional_features = dev_df_train[['text_length', 'roberta_sent_neg', 'roberta_sent_mixed',
                            'roberta_sent_pos', 'names', 'organizations', 'dates',
                            'count_tokens', 'cleaned_text_length', 'punctuation_count']].to_numpy()

# embeddings = np.vstack(new_df['embedding'])

In [ ]:
dev_df_train['converted_embedding'] = dev_df_train['embedding'].apply(convert_to_numpy_array)

In [ ]:
# pca = PCA(n_components = 50)
dev_reduced_embeddings = pca.transform(dev_df_train['converted_embedding'].to_list())

In [ ]:
dev_embeddings_np = np.array(dev_reduced_embeddings)
dev_tfidf_features_np = dev_tfidf_features.toarray()
dev_additional_features_np = np.array(dev_additional_features)

In [ ]:
X_train = np.hstack([dev_tfidf_features_np,dev_embeddings_np, dev_additional_features_np])
y_train = dev_df_train['label']

In [ ]:
# X_train, X_test, y_train,y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)

# clf = RandomForestClassifier()
# clf.fit(X_train,y_train)

In [ ]:
# X_train.shape
X_train.shape

(1026, 4666)

In [ ]:
# dataset are big, take time to process - reducing the data size
# from sklearn.utils import resample

In [ ]:
# X_train_sample, y_train_sample = resample(X_train, y_train, n_samples=5000, random_state=42)

In [ ]:
# redusing the dimensionality
# pca = PCA(n_components = 500)
# X_train_sample_reduced = pca.fit_transform(X_train_sample)

## 5/2/25
# model : RFC
# HyperParameter tuning on dev data

In [ ]:
rf = RandomForestClassifier(random_state = 42)

In [ ]:
print("Parameters are currently in used:\n")
pprint(rf.get_params())

Parameters are currently in used:

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


In [ ]:
# # Number of trees in random forest
# n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# # Number of features to consider at every split
# max_features = ['log2', 'sqrt']
# # Maximum number of levels in tree
# max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
# max_depth.append(None)
# # Minimum number of samples required to split a node
# min_samples_split = [2, 5, 10]
# # Minimum number of samples required at each leaf node
# min_samples_leaf = [1, 2, 4]
# # Method of selecting samples for training each tree
# bootstrap = [True, False]
# # Create the random grid - parameter distributions
# random_grid = {'n_estimators': n_estimators,
#                'max_features': max_features,
#                'max_depth': max_depth,
#                'min_samples_split': min_samples_split,
#                'min_samples_leaf': min_samples_leaf,
#                'bootstrap': bootstrap}
rfc_param_dist = {
    'n_estimators': [int(x) for x in np.linspace(start=100, stop=2000, num=10)],  # Trees
    'max_features': ['sqrt', 'log2'],  # Features per split
    'max_depth': [int(x) for x in np.linspace(10, 110, num=11)] + [None],  # Depth
    'min_samples_split': [2, 5, 10],  # Min samples for split
    'min_samples_leaf': [1, 2, 4],  # Min samples per leaf
    'bootstrap': [True, False],  # Bootstrap sampling
    'class_weight': [None, 'balanced'],  # Handle imbalance
    'criterion': ['gini', 'entropy'],  # Splitting strategy
    'ccp_alpha': [0.0, 0.01, 0.1],  # Post-pruning
    'min_impurity_decrease': [0.0, 0.01, 0.1]  # Stop unhelpful splits
}

pprint(rfc_param_dist)

{'bootstrap': [True, False],
 'ccp_alpha': [0.0, 0.01, 0.1],
 'class_weight': [None, 'balanced'],
 'criterion': ['gini', 'entropy'],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['sqrt', 'log2'],
 'min_impurity_decrease': [0.0, 0.01, 0.1],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [100, 311, 522, 733, 944, 1155, 1366, 1577, 1788, 2000]}


In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation,
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = rfc_param_dist, n_iter = 50, cv = 5, verbose=2, random_state=42, n_jobs = -1, scoring = 'f1')
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_iter=50,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'ccp_alpha': [0.0, 0.01, 0.1],
                                        'class_weight': [None, 'balanced'],
                                        'criterion': ['gini', 'entropy'],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['sqrt', 'log2'],
                                        'min_impurity_decrease': [0.0, 0.01,
                                                                  0.1],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [100, 311, 522, 733,
                                                         944, 1155, 1366, 1577,
                                                         1788, 2000]},
                   random_state=42, scoring='f1', verbose=2)

In [ ]:
pprint(rf_random.best_params_)



{'bootstrap': False,
 'ccp_alpha': 0.01,
 'class_weight': 'balanced',
 'criterion': 'entropy',
 'max_depth': 30,
 'max_features': 'sqrt',
 'min_impurity_decrease': 0.01,
 'min_samples_leaf': 4,
 'min_samples_split': 2,
 'n_estimators': 1788}


In [ ]:
rf_random.best_params_

{'n_estimators': 400,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': None,
 'bootstrap': False}

In [ ]:
rf_random.best_score_

0.47258390726438193

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# # hyperparameter tuning
# # n_estimators = [int(x) for x in np.linspace(start = 200, stop = 1000, num = 10)] # number of trees in random forest
# n_estimators = [100, 200, 300] # number of trees in random forest
# max_features = ['auto', 'sqrt'] # no. of features to consider at every split
# # max_depth = [int(x) for x in np.linspace(10, 50, num = 11)] # max number of levels in tree
# max_depth = [None, 10, 20, 30] # max number of levels in tree
# # max_depth.append(None)
# min_samples_split = [2,5,10]# minimum number of samples required to split node
# min_samples_leaf = [1, 2, 4] # min number of samples required at each leaf node
# bootstarp = [True, False] # method of selecting samples for training each tree

# # create the random grid
# random_grid = {
#     'n_estimators': n_estimators,
#     'max_features': max_features,
#     'max_depth': max_depth,
#     'min_samples_split': min_samples_split,
#     'min_samples_leaf': min_samples_leaf,
#     'bootstrap': bootstarp
# }
# pprint(random_grid)


### Taking time to tune

In [ ]:
# # use the random grid to search for best hyperparameters
# # first create the base model to tune
# rf = RandomForestRegressor()
# # random search of parameters, using 3 fold cross validation
# # search across 100 different combinations, and use all availiable cores
# rf_random = RandomizedSearchCV(
#                             estimator = rf,
#                             param_distributions = random_grid,
#                             n_iter = 50, #  for large dataset 50 to 75
#                             cv = 3,
#                             verbose = 2,
#                             random_state = 42,
#                             n_jobs = -1 # using all processors
#                             )
# # fit the random search model
# rf_random.fit(X_train, y_train)

In [ ]:
# rf_random.best_params_

### Model : XGB
## technique : Grid Search CV
### Hyperparameter Tuning

In [ ]:
# print(sklearn.__version__)
# print(xgboost.__version__)

In [ ]:
# !pip uninstall -y scikit-learn
# !pip install scikit-learn==1.3.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 81.0 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imbalanced-learn 0.13.0 requires scikit-learn<2,>=1.3.2, but you have scikit-learn 1.3.1 which is incompatible.


In [ ]:
xgb_model = XGBClassifier()
pprint(xgb_model.get_params)

<bound method XGBModel.get_params of XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)>


In [ ]:
# max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
# n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# param_dist = {
#     'max_depth': max_depth,
#     'learning_rate': stats.uniform(0.01,0.1),
#     'subsample': stats.uniform(0.5,0.5),
#     'n_estimators': n_estimators
# }
xgb_param_dist = {
  'n_estimators': [int(x) for x in np.linspace(start = 200, stop = 2000, num = 15)],  # Number of boosting rounds
    'max_depth': [int(x) for x in np.linspace(10, 110, num = 11)],  # Depth of trees
    'learning_rate': [0.01, 0.05, 0.1, 0.2],  # Step size shrinkage
    'subsample': [0.6, 0.8, 1.0],  # Fraction of samples used per tree
    'colsample_bytree': [0.6, 0.8, 1.0],  # Fraction of features used per tree
    'gamma': [0, 0.1, 0.2, 0.5, 1.0],  # Minimum loss reduction to split a node
    'min_child_weight': [1, 3, 5, 7],  # Minimum sum of weights in a leaf
    'reg_alpha': [0, 0.01, 0.1, 1, 10],  # L1 regularization (Lasso)
    'reg_lambda': [0, 0.01, 0.1, 1, 10],  # L2 regularization (Ridge)
    'scale_pos_weight': [1, 2, 5, 10, 20],  # Balance positive class in imbalanced data
    'objective': ['binary:logistic'],  # Binary classification
    'eval_metric': ['logloss', 'auc', 'error']  # Evaluation metrics
}


pprint(xgb_param_dist)

{'colsample_bytree': [0.6, 0.8, 1.0],
 'eval_metric': ['logloss', 'auc', 'error'],
 'gamma': [0, 0.1, 0.2, 0.5, 1.0],
 'learning_rate': [0.01, 0.05, 0.1, 0.2],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110],
 'min_child_weight': [1, 3, 5, 7],
 'n_estimators': [200,
                  328,
                  457,
                  585,
                  714,
                  842,
                  971,
                  1100,
                  1228,
                  1357,
                  1485,
                  1614,
                  1742,
                  1871,
                  2000],
 'objective': ['binary:logistic'],
 'reg_alpha': [0, 0.01, 0.1, 1, 10],
 'reg_lambda': [0, 0.01, 0.1, 1, 10],
 'scale_pos_weight': [1, 2, 5, 10, 20],
 'subsample': [0.6, 0.8, 1.0]}


In [ ]:
xgb_random = RandomizedSearchCV(xgb_model, xgb_param_dist, n_iter = 50, cv=5, scoring='f1', verbose=2, random_state=42, n_jobs = -1)
xgb_random.fit(X_train, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate...
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110],
                                        'min_child_weight': [1, 3, 5, 7],
                                        'n_estimators': [200, 328, 457, 585,
                                                         714, 842, 971, 1100,
                                                         1228, 1357, 1485, 1614,
                                                         1742, 1871, 2000],
                                        'objective': ['binary:logistic'],
                                        'reg_alpha': [0, 0.01, 0.1, 1, 10],
                                        'reg_lambda': [0, 0.01, 0.1, 1, 10],
                                        'scale_pos_weight': [1, 2, 5, 10, 20],
                                        'subsample': [0.6, 0.8, 1.0]},
                   random_state=42, scoring='f1', verbose=2)

In [ ]:
pprint(xgb_random.best_params_)


{'colsample_bytree': 1.0,
 'eval_metric': 'logloss',
 'gamma': 0,
 'learning_rate': 0.05,
 'max_depth': 30,
 'min_child_weight': 5,
 'n_estimators': 1742,
 'objective': 'binary:logistic',
 'reg_alpha': 0,
 'reg_lambda': 0.01,
 'scale_pos_weight': 5,
 'subsample': 0.6}


### Hyperparameter tuning for Decision Tree Classifier
- improve performance
- Reduces Overfitting (trees complexity - max_depth, preventing excessive granilarity - min_samples_split)
- enhanced generalization - preform well on unseen data
- addressing class imbalance - min_weight_fraction_leaf
- Tiloring the model for specific task

### Hyperparameters:
- criteria
- max_depth
- min_samples_split
- min_samples_leaf
- max_features
- min_weight_fraction_leaf

### Methods for hp tuning

- Grid search
- Randomized search
- Bayesian optimization


In [ ]:
decision_tree_classifier = DecisionTreeClassifier(random_state = 42)

In [ ]:
print("Parameters are currently in used:\n")
pprint(decision_tree_classifier.get_params())

Parameters are currently in used:

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'random_state': 42,
 'splitter': 'best'}


In [ ]:
param_dist = {
    'criterion': ['gini', 'entropy', 'log_loss'],  # Try both impurity measures
    'splitter': ['best', 'random'],  # 'random' may help generalization
    'max_depth': [5, 10, 15, None],  # Control tree size (None = full depth) # randint(1, 20)
    'min_samples_split': [2, 5, 10, 20],  # Minimum samples to split a node
    'min_samples_leaf': [1, 2, 5, 10],  # Minimum samples per leaf node
    'max_features': ['sqrt', 'log2', None],  # Feature selection at each split
    'class_weight': [None, 'balanced'],  # Handle class imbalance
    'ccp_alpha': [0.0, 0.01, 0.1],  # Pruning parameter (0 = no pruning)
    'max_leaf_nodes': [None, 10, 20, 50]  # Limit the number of leaf nodes
}
pprint(param_dist)

{'ccp_alpha': [0.0, 0.01, 0.1],
 'class_weight': [None, 'balanced'],
 'criterion': ['gini', 'entropy', 'log_loss'],
 'max_depth': [5, 10, 15, None],
 'max_features': ['sqrt', 'log2', None],
 'max_leaf_nodes': [None, 10, 20, 50],
 'min_samples_leaf': [1, 2, 5, 10],
 'min_samples_split': [2, 5, 10, 20],
 'splitter': ['best', 'random']}


In [ ]:
rf_dct = RandomizedSearchCV(estimator=decision_tree_classifier, param_distributions=param_dist, n_jobs = -1, n_iter=50, cv=5, scoring='f1', verbose = 2, random_state=42)

pprint(rf_dct.get_params())

{'cv': 5,
 'error_score': nan,
 'estimator': DecisionTreeClassifier(random_state=42),
 'estimator__ccp_alpha': 0.0,
 'estimator__class_weight': None,
 'estimator__criterion': 'gini',
 'estimator__max_depth': None,
 'estimator__max_features': None,
 'estimator__max_leaf_nodes': None,
 'estimator__min_impurity_decrease': 0.0,
 'estimator__min_samples_leaf': 1,
 'estimator__min_samples_split': 2,
 'estimator__min_weight_fraction_leaf': 0.0,
 'estimator__monotonic_cst': None,
 'estimator__random_state': 42,
 'estimator__splitter': 'best',
 'n_iter': 50,
 'n_jobs': -1,
 'param_distributions': {'ccp_alpha': [0.0, 0.01, 0.1],
                         'class_weight': [None, 'balanced'],
                         'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': [5, 10, 15, None],
                         'max_features': ['sqrt', 'log2', None],
                         'max_leaf_nodes': [None, 10, 20, 50],
                         'min_samples_leaf': [1, 2, 5, 1

In [ ]:
rf_dct.fit(X_train, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


RandomizedSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'ccp_alpha': [0.0, 0.01, 0.1],
                                        'class_weight': [None, 'balanced'],
                                        'criterion': ['gini', 'entropy',
                                                      'log_loss'],
                                        'max_depth': [5, 10, 15, None],
                                        'max_features': ['sqrt', 'log2', None],
                                        'max_leaf_nodes': [None, 10, 20, 50],
                                        'min_samples_leaf': [1, 2, 5, 10],
                                        'min_samples_split': [2, 5, 10, 20],
                                        'splitter': ['best', 'random']},
                   random_state=42, scoring='f1', verbose=2)

In [ ]:
pprint(rf_dct.best_params_)

{'ccp_alpha': 0.01,
 'class_weight': 'balanced',
 'criterion': 'log_loss',
 'max_depth': 10,
 'max_features': None,
 'max_leaf_nodes': 10,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'splitter': 'best'}


### Hyperparameter tuning for Logistic Regression

In [ ]:
lr = LogisticRegression(random_state=42)

In [ ]:
lr_param_dist = {
   'penalty':['l1', 'l2', 'elasticnet', None] ,
   'dual':[True, False],

}

### Hyperparameter tuning for KNN

In [ ]:
knn = KNeighborsClassifier()

In [ ]:
pprint(knn.get_params())

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}


In [ ]:
knn_param_dist = {
    'n_neighbors': list(range(3, 51, 2)),  # Odd values to avoid ties
    'weights': ['uniform', 'distance'],  # Distance-based weighting can help
    'metric': ['euclidean', 'manhattan', 'minkowski'],  # Different distance metrics
    'p': [1, 2]  # p=1 (Manhattan), p=2 (Euclidean) for Minkowski distance
}

pprint(knn_param_dist)

{'metric': ['euclidean', 'manhattan', 'minkowski'],
 'n_neighbors': [3,
                 5,
                 7,
                 9,
                 11,
                 13,
                 15,
                 17,
                 19,
                 21,
                 23,
                 25,
                 27,
                 29,
                 31,
                 33,
                 35,
                 37,
                 39,
                 41,
                 43,
                 45,
                 47,
                 49],
 'p': [1, 2],
 'weights': ['uniform', 'distance']}


In [ ]:
rcv_knn = RandomizedSearchCV(estimator=knn, param_distributions=knn_param_dist, n_iter=50, n_jobs = -1, cv = 5, verbose =2 , random_state = 42, scoring= 'f1')
pprint(rcv_knn.get_params())

{'cv': 5,
 'error_score': nan,
 'estimator': KNeighborsClassifier(),
 'estimator__algorithm': 'auto',
 'estimator__leaf_size': 30,
 'estimator__metric': 'minkowski',
 'estimator__metric_params': None,
 'estimator__n_jobs': None,
 'estimator__n_neighbors': 5,
 'estimator__p': 2,
 'estimator__weights': 'uniform',
 'n_iter': 50,
 'n_jobs': -1,
 'param_distributions': {'metric': ['euclidean', 'manhattan', 'minkowski'],
                         'n_neighbors': [3,
                                         5,
                                         7,
                                         9,
                                         11,
                                         13,
                                         15,
                                         17,
                                         19,
                                         21,
                                         23,
                                         25,
                                         27,


In [ ]:
rcv_knn.fit(X_train, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


RandomizedSearchCV(cv=5, estimator=KNeighborsClassifier(), n_iter=50, n_jobs=-1,
                   param_distributions={'metric': ['euclidean', 'manhattan',
                                                   'minkowski'],
                                        'n_neighbors': [3, 5, 7, 9, 11, 13, 15,
                                                        17, 19, 21, 23, 25, 27,
                                                        29, 31, 33, 35, 37, 39,
                                                        41, 43, 45, 47, 49],
                                        'p': [1, 2],
                                        'weights': ['uniform', 'distance']},
                   random_state=42, scoring='f1', verbose=2)

In [ ]:
pprint(rcv_knn.best_params_)

{'metric': 'minkowski', 'n_neighbors': 3, 'p': 2, 'weights': 'distance'}


### Hyperparameter tuning for AdaBoost

In [ ]:
best_params = {
  'ccp_alpha': 0.01,
 'class_weight': 'balanced',
 'criterion': 'log_loss',
 'max_depth': 10,
 'max_features': None,
 'max_leaf_nodes': 10,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'splitter': 'best'
}
base_estimetor = DecisionTreeClassifier(**best_params, random_state=42)


In [ ]:
adb = AdaBoostClassifier(estimator=base_estimetor, random_state = 42)
adb_param_dist = {
    'n_estimators': randint(50, 300),  # Number of weak learners
    'learning_rate': uniform(0.01, 1.0),  # Learning rate
    #'base_estimator__max_depth': randint(1, 5),  # Depth of weak learners
}

In [ ]:
rf_adb = RandomizedSearchCV(
    estimator=adb,
    param_distributions=adb_param_dist,
    n_iter=50,  # Number of parameter combinations to try
    cv=5,       # 5-fold cross-validation
    scoring='f1',  # Metric to optimize (F1 score for binary classification)
    verbose=2,   # Print progress
    random_state=42,
    n_jobs=-1    # Use all available cores
)
rf_adb.fit(X_train, y_train)
pprint(rf_adb.best_params_)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
{'learning_rate': 0.34370861113902185, 'n_estimators': 153}


### Hyperparameter tuning for Gradient Boosting

In [ ]:
gdb = GradientBoostingClassifier(random_state=42)

In [ ]:
gdb_param_dist = {
    'n_estimators': randint(50, 300),  # Number of boosting stages
    'learning_rate': uniform(0.01, 0.3),  # Learning rate
    'max_depth': randint(3, 10),  # Maximum depth of trees
    'min_samples_split': randint(2, 20),  # Minimum samples to split
    'min_samples_leaf': randint(1, 20),  # Minimum samples at a leaf
    'max_features': ['sqrt', 'log2', None],  # Number of features to consider
    'subsample': uniform(0.5, 0.5),  # Fraction of samples used
}

In [ ]:
rf_gdb = RandomizedSearchCV(estimator=gdb, param_distributions=gdb_param_dist, n_iter=50, n_jobs = -1, cv = 5, verbose =2 , random_state = 42, scoring= 'f1')
rf_gdb.fit(X_train, y_train)
print(rf_gdb.best_params_)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
{'learning_rate': 0.27616382277289797, 'max_depth': 3, 'max_features': None, 'min_samples_leaf': 5, 'min_samples_split': 16, 'n_estimators': 255, 'subsample': 0.9244569121330419}


In [ ]:
pprint(rf_gdb.best_params_)

{'learning_rate': 0.27616382277289797,
 'max_depth': 3,
 'max_features': None,
 'min_samples_leaf': 5,
 'min_samples_split': 16,
 'n_estimators': 255,
 'subsample': 0.9244569121330419}


### Hyperparameter tuning for LightGBM

In [ ]:
lgbm = LGBMClassifier(objective='binary', metric='binary_logloss', verbose=-1, random_state=42)

In [ ]:
lgbm_params_dist = {
    'num_leaves': randint(20, 100),  # Number of leaves
    'max_depth': randint(3, 15),     # Maximum depth
    'learning_rate': uniform(0.01, 0.3),  # Learning rate
    'n_estimators': randint(50, 300),  # Number of boosting rounds
    'lambda_l1': uniform(0, 1),  # L1 regularization
    'lambda_l2': uniform(0, 1),  # L2 regularization
    'min_child_samples': randint(10, 50),  # Minimum data in a leaf
    'feature_fraction': uniform(0.5, 0.5),  # Feature fraction
    'bagging_fraction': uniform(0.5, 0.5),  # Bagging fraction
    'bagging_freq': randint(0, 10),  # Bagging frequency
}

In [ ]:
rf_lgbm = RandomizedSearchCV(estimator=lgbm, param_distributions=lgbm_params_dist, n_iter=50, n_jobs = -1, cv = 5, verbose =2 , random_state = 42, scoring= 'f1')
rf_lgbm.fit(X_train, y_train)
pprint(rf_lgbm.best_params_)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


{'bagging_fraction': 0.856135294962221,
 'bagging_freq': 4,
 'feature_fraction': 0.6424202471887338,
 'lambda_l1': 0.036886947354532795,
 'lambda_l2': 0.6095643339798968,
 'learning_rate': 0.16080370696865845,
 'max_depth': 8,
 'min_child_samples': 32,
 'n_estimators': 278,
 'num_leaves': 20}


## tuning parameters for different sampling techniques

In [ ]:
df_train_org =  pd.read_csv('/content/drive/MyDrive/Thesis/data/train/new_11.csv')
data_to_merge = pd.read_csv('/content/drive/MyDrive/Thesis/data/train/ner_features_with_relevent_noise_prompt.csv')
vectorizer_path = '/content/drive/MyDrive/Thesis/models/tfidf_vectorizer_4300.pkl'
tfidf_vect = joblib.load(vectorizer_path)

pca_path = '/content/drive/MyDrive/Thesis/models/pca_356.pkl'
pca = joblib.load(pca_path)

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 1.5.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator TfidfVectorizer from version 1.5.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator PCA from version 1.5.2 when using version 1.6.1. This might lead to breaking code or invalid 

In [ ]:
df_train_org.drop('Unnamed: 0', axis = 1, inplace = True)
data_to_merge.drop('Unnamed: 0', axis = 1, inplace = True)

In [ ]:
# 1. drop 9896 data from NO class
# 2. drop all NONE for all classes
after_drop = data_to_merge.drop(data_to_merge[(data_to_merge['class_label'] == 'No') & (data_to_merge['NONE'] == True)].index)
# after_drop = data_to_merge.drop(data_to_merge[(data_to_merge['NONE'] == True)].index)
after_drop.shape

(12605, 24)

In [ ]:
merged_data = pd.merge(df_train_org, after_drop, left_on='Sentence_id', right_on='Sentence_id', how='inner')
merged_data.drop(['Text_y', 'class_label_y','CARDINAL', 'DATE', 'EVENT', 'FAC', 'GPE', 'LANGUAGE', 'LAW', 'LOC','MONEY', 'NORP', 'ORDINAL', 'ORG', 'PERCENT', 'PERSON', 'PRODUCT','QUANTITY', 'TIME', 'WORK_OF_ART', 'NONE', 'Noise_text','Relevant_text'], axis = 1, inplace = True)
merged_data.rename(columns={'Text_x': 'Text', 'class_label_x': 'class_label'}, inplace=True)
merged_data.columns

Index(['Sentence_id', 'Text', 'class_label', 'Tokens', 'text_length',
       'sentiment_score_veda', 'sentiment_score_textblob',
       'sentiment_score_bert', 'sentiment_score_roberta', 'bert_sent_neg',
       'bert_sent_pos', 'roberta_sent_neg', 'roberta_sent_neut',
       'roberta_sent_mixed', 'roberta_sent_pos', 'labels', 'names',
       'organizations', 'locations', 'dates', 'verbs', 'action_verbs',
       'filtered_action_verbs', 'joined_tokens', 'count_verb',
       'count_action_verb', 'count_filtered_action_verb', 'count_tokens',
       'cleaned_text', 'cleaned_text_length', 'contains_question_mark',
       'contains_exclamation', 'contains_ellipsis', 'num_exclamations',
       'num_questions', 'num_ellipses', 'punctuation_count',
       'converted_embedding', 'text-tfidf'],
      dtype='object')

In [ ]:
merged_data = merged_data.dropna()

In [ ]:
# for handling empty Tokens
def handle_empty_tokens(row):
  if row['Tokens'] is np.nan:
      return row['Text']

  else:
    return ' '.join(row['Tokens'])

In [ ]:
# df_train_org['converted_embedding'].iloc[0]
def convert_to_numpy_array(embd_str):
    #print(f"string: {embd_str}")
    #print(type(embd_str))
    embd_str_clean = embd_str.replace("\n", " ").replace(" ", " ")
    #print(type(embd_str_clean))
    #print(f"string cleansing: {embd_str_clean}")
    embd_str_clean = embd_str_clean.strip('[]')
    #print(type(embd_str_clean))
    #print(f"removing the outer brackets {embd_str_clean}")
    embd_list = embd_str_clean.split()
    #print(f"Split the string into individual values {embd_list}")
    embed_array = np.array(embd_list, dtype = float)
    #print(f"list to array {embed_array}")
    return embed_array

In [ ]:
# df_train_org['converted_embedding'] = df_train_org['converted_embedding'].apply(convert_to_numpy_array)
merged_data['converted_embedding'] = merged_data['converted_embedding'].apply(convert_to_numpy_array)
# pca = PCA(n_components = 356)
train_reduced_embeddings = pca.transform(merged_data['converted_embedding'].to_list())

train_text = merged_data['text-tfidf']
# text = df_train_org['text-tfidf']
# vectorizer_path = '/content/drive/MyDrive/Thesis/models/tfidf_vectorizer_4300.pkl'
# tfidf_vect = joblib.load(vectorizer_path)
tfidf_features = tfidf_vect.transform(train_text)

train_additional_features = merged_data[['text_length', 'roberta_sent_neg', 'roberta_sent_mixed',
                            'roberta_sent_pos', 'names', 'organizations', 'dates',
                            'count_tokens', 'cleaned_text_length', 'punctuation_count']].to_numpy()

train_embeddings_np = np.array(train_reduced_embeddings)
train_tfidf_features_np = tfidf_features.toarray()
train_additional_features_np = np.array(train_additional_features)

X_train = np.hstack([train_tfidf_features_np,train_embeddings_np, train_additional_features_np])
y_train = merged_data['labels']

In [ ]:
X_train.shape

(12597, 4666)

### param distribution for cnn, rus, enn, tmk, allknn, renn, iht, nm, ncr, os (UNDER SAMPLING)

In [ ]:
rfc_best_params = {
  'bootstrap': False,
 'ccp_alpha': 0.01,
 'class_weight': 'balanced',
 'criterion': 'entropy',
 'max_depth': 30,
 'max_features': 'sqrt',
 'min_impurity_decrease': 0.01,
 'min_samples_leaf': 4,
 'min_samples_split': 2,
 'n_estimators': 1788
}
xgb_best_params = {
 'colsample_bytree': 1.0,
 'eval_metric': 'logloss',
 'gamma': 0,
 'learning_rate': 0.05,
 'max_depth': 30,
 'min_child_weight': 5,
 'n_estimators': 1742,
 'objective': 'binary:logistic',
 'reg_alpha': 0,
 'reg_lambda': 0.01,
 'scale_pos_weight': 5,
 'subsample': 0.6
}

In [ ]:
param_distribution_us = {
    'RandomUnderSampler': {
        'sampling_strategy': ['majority', 'not minority', 'not majority', 'all','auto'],
        'random_state': [42]
    },
    'EditedNearestNeighbours': {
         'sampling_strategy': ['majority', 'not minority', 'not majority', 'all', 'auto'],
        # 'random_state': [42],
        'n_jobs': [-1],
        'n_neighbors': [3, 4, 5],
        'kind_sel': ['all', 'mode']
    },
    'CondensedNearestNeighbour': {
        'sampling_strategy': ['majority', 'not minority', 'not majority', 'all', 'auto'],
        'random_state': [42],
        'n_jobs': [-1],
        'n_neighbors': [3, 4, 5],
        'n_seeds_S':[1, 5, 10]
    },
    'TomekLinks': {
        'sampling_strategy': ['majority', 'not minority', 'not majority', 'all', 'auto'],
        'n_jobs': [-1]
    },
    'ClusterCentroids': {
        'sampling_strategy': ['majority', 'not minority', 'not majority', 'all', 'auto'],
        'random_state':[42],
        'voting': ['hard', 'soft', 'auto']
    },
    'RepeatedEditedNearestNeighbours': {
        'sampling_strategy': ['majority', 'not minority', 'not majority', 'all', 'auto'],
        'random_state':[42],
        'n_jobs': [-1],
        'kind_sel': ['all', 'mode'],
        'n_neighbors': [3, 4, 5],
        'max_iter': [50, 100, 200]
    },
    'NearMiss': {
        'sampling_strategy': ['majority', 'not minority', 'not majority', 'all', 'auto'],
        'version': [1, 2, 3],
        'n_neighbors': [3, 4,  5],
        'n_neighbors_ver3':[3, 4, 5],
        'n_jobs': [-1]
    },
    'NeighbourhoodCleaningRule': {
        'sampling_strategy': ['majority', 'not minority', 'not majority', 'all', 'auto'],
        'edited_nearest_neughbors': [None, 'all', 'mode'],
        'n_neighbors': [3, 4, 5],
        'n_jobs': [-1]
    },
    'AllKNN': {
        'sampling_strategy': ['majority', 'not minority', 'not majority', 'all', 'auto'],
        'n_neighbors': [3, 4, 5],
        'kind_sel': ['all', 'mode'],
        'allow_minority': [True, False],
        'n_jobs': [-1]
    },
    'OneSidedSelection': {
        'sampling_strategy': ['majority', 'not minority', 'not majority', 'all', 'auto'],
        'n_seeds_S':[1,5,10],
        'n_jobs': [-1],
        'n_neighbors': [3, 4, 5],
        'random_state': [42]
    },
    'InstanceHardnessThreshold': {
        'estimator': XGBClassifier(**xgb_best_params),
        'cv': [3, 5, 10],
        'sampling_strategy': ['majority', 'not minority', 'not majority', 'all', 'auto'],
        'n_jobs': [-1]
    }

}

In [ ]:
# RandomUnderSampler, EditedNearestNeighbours, CondensedNearestNeighbour, TomekLinks, ClusterCentroids, RepeatedEditedNearestNeighbours, NearMiss, NeighbourhoodCleaningRule, OneSidedSelection, AllKNN, InstanceHardnessThreshold
# ('classifier_2', RandomForestClassifier(**rfc_best_params,random_state=42))
                                    # ('classifier_3', XGBClassifier())
                                    # ('classifier_4', RandomForestClassifier(**xgb_best_params,random_state=42))
pipeline = {
    # 'RandomUnderSampler_rfc': Pipeline([('sampler', RandomUnderSampler()),
    #                                 ('classifier', RandomForestClassifier(random_state=42)),
    #                                 ]),
    # # # 'RandomUnderSampler_rfc_pt': Pipeline([('sampler', RandomUnderSampler()),
    # # #                                 ('classifier', RandomForestClassifier(**rfc_best_params,random_state=42)),
    # #                                 ]),
    # 'RandomUnderSampler_xgb': Pipeline([('sampler', RandomUnderSampler()),
    #                                 ('classifier', XGBClassifier()),
    #                                 ]),
    # 'RandomUnderSampler_xgb_pt': Pipeline([('sampler', RandomUnderSampler()),
    #                                 ('classifier', XGBClassifier(**xgb_best_params)),
    #                                 ]),
    # 'EditedNearestNeighbours_rfc': Pipeline([('sampler', EditedNearestNeighbours()),
    #                                 ('classifier', RandomForestClassifier(random_state=42))]),
    # # 'EditedNearestNeighbours_rfc_pt': Pipeline([('sampler', EditedNearestNeighbours()),
    # #                                 ('classifier', RandomForestClassifier(**rfc_best_params,random_state=42))]),
    # 'EditedNearestNeighbours_xgb': Pipeline([('sampler', EditedNearestNeighbours()),
    #                                 ('classifier', XGBClassifier())]),
    # 'EditedNearestNeighbours_xgb_pt': Pipeline([('sampler', EditedNearestNeighbours()),
    #                                 ('classifier', XGBClassifier(**xgb_best_params))]),

    'CondensedNearestNeighbour_rfc': Pipeline([('sampler', CondensedNearestNeighbour()),
                                    ('classifier', RandomForestClassifier(random_state=42))]),
    # 'CondensedNearestNeighbour_rfc_pt': Pipeline([('sampler', CondensedNearestNeighbour()),
    #                                 ('classifier', RandomForestClassifier(**rfc_best_params,random_state=42))]),
    # 'CondensedNearestNeighbour_xgb': Pipeline([('sampler', CondensedNearestNeighbour()),
    #                                 ('classifier', XGBClassifier())]),
    #  'CondensedNearestNeighbour_xgb_pt': Pipeline([('sampler', CondensedNearestNeighbour()),
    #                                 ('classifier', XGBClassifier(**xgb_best_params))]),


    'TomekLinks_rfc': Pipeline([('sampler', TomekLinks()),
                                    ('classifier', RandomForestClassifier(random_state=42))]),
    # 'TomekLinks_rfc_pt': Pipeline([('sampler', TomekLinks()),
    #                                 ('classifier', RandomForestClassifier(**rfc_best_params, random_state=42))]),
    # 'TomekLinks_xgb': Pipeline([('sampler', TomekLinks()),
    #                                 ('classifier', XGBClassifier())]),
    # 'TomekLinks_xgb_pt': Pipeline([('sampler', TomekLinks()),
    #                                 ('classifier', XGBClassifier(**xgb_best_params))]),

    'ClusterCentroids_rfc': Pipeline([('sampler', ClusterCentroids()),
                                    ('classifier', RandomForestClassifier(random_state=42))]),'ClusterCentroids_rfc': Pipeline([('sampler', ClusterCentroids()),
                                    ('classifier', RandomForestClassifier(random_state=42))]),
    # 'ClusterCentroids_rfc_pt': Pipeline([('sampler', ClusterCentroids()),
    #                                 ('classifier', RandomForestClassifier(**rfc_best_params, random_state=42))]),
    # 'ClusterCentroids_xgb': Pipeline([('sampler', ClusterCentroids()),
    #                                 ('classifier', XGBClassifier())]),
    # 'ClusterCentroids_xgfb_pt': Pipeline([('sampler', ClusterCentroids()),
    #                                 ('classifier', XGBClassifier(**xgb_best_params))]),

    'RepeatedEditedNearestNeighbours_rfc': Pipeline([('sampler', RepeatedEditedNearestNeighbours()),
                                    ('classifier', RandomForestClassifier(random_state=42))]),
    # 'RepeatedEditedNearestNeighbours_rfc_pt': Pipeline([('sampler', RepeatedEditedNearestNeighbours()),
    #                                 ('classifier', RandomForestClassifier(**rfc_best_params, random_state=42))]),
    # 'RepeatedEditedNearestNeighbours_xgb': Pipeline([('sampler', RepeatedEditedNearestNeighbours()),
    #                                 ('classifier', XGBClassifier())]),
    # 'RepeatedEditedNearestNeighbours_xgb_pt': Pipeline([('sampler', RepeatedEditedNearestNeighbours()),
    #                                 ('classifier', XGBClassifier(**xgb_best_params))]),

    'NearMiss_rfc': Pipeline([('sampler', NearMiss()),
                                    ('classifier', RandomForestClassifier(random_state=42))]),
    # 'NearMiss_rfc_pt': Pipeline([('sampler', NearMiss()),
    #                                 ('classifier', RandomForestClassifier(**rfc_best_params, random_state=42))]),
    # 'NearMiss_xgb': Pipeline([('sampler', NearMiss()),
    #                                 ('classifier',XGBClassifier())]),
    # 'NearMiss_xgb_pt': Pipeline([('sampler', NearMiss()),
    #                                 ('classifier', XGBClassifier(**xgb_best_params))]),

    'NeighbourhoodCleaningRule_rfc': Pipeline([('sampler', NeighbourhoodCleaningRule()),
                                    ('classifier', RandomForestClassifier(random_state=42))]),
    # 'NeighbourhoodCleaningRule_rfc_pt': Pipeline([('sampler', NeighbourhoodCleaningRule()),
    #                                 ('classifier', RandomForestClassifier(**rfc_best_params, random_state=42))]),
    # 'NeighbourhoodCleaningRule_xgb': Pipeline([('sampler', NeighbourhoodCleaningRule()),
    #                                 ('classifier', XGBClassifier())]),
    # 'NeighbourhoodCleaningRule_xgb_pt': Pipeline([('sampler', NeighbourhoodCleaningRule()),
    #                                 ('classifier', XGBClassifier(**xgb_best_params))]),

    'AllKNN_rfc': Pipeline([('sampler', AllKNN()),
                                    ('classifier', RandomForestClassifier(random_state=42))]),
    # 'AllKNN_rfc_pt': Pipeline([('sampler', AllKNN()),
    #                                 ('classifier', RandomForestClassifier(**rfc_best_params,random_state=42))]),
    # 'AllKNN_xgb': Pipeline([('sampler', AllKNN()),
    #                                 ('classifier', XGBClassifier())]),
    # 'AllKNN_xgb_pt': Pipeline([('sampler', AllKNN()),
    #                                 ('classifier', XGBClassifier(**xgb_best_params))]),

    'OneSidedSelection_rfc': Pipeline([('sampler', OneSidedSelection()),
                                    ('classifier', RandomForestClassifier(random_state=42))]),
    # 'OneSidedSelection_rfc_pt': Pipeline([('sampler', OneSidedSelection()),
    #                                 ('classifier', RandomForestClassifier(**rfc_best_params, random_state=42))]),
    # 'OneSidedSelection_xgb': Pipeline([('sampler', OneSidedSelection()),
    #                                 ('classifier', XGBClassifier())]),
    # 'OneSidedSelection_xgb_pt': Pipeline([('sampler', OneSidedSelection()),
    #                                 ('classifier', XGBClassifier(**xgb_best_params))]),

    'InstanceHardnessThreshold_rfc': Pipeline([('sampler', InstanceHardnessThreshold()),
                                    ('classifier', RandomForestClassifier(random_state=42))]),
    # 'InstanceHardnessThreshold_rfc_pt': Pipeline([('sampler', InstanceHardnessThreshold()),
    #                                 ('classifier', RandomForestClassifier(**rfc_best_params,random_state=42))]),
    # 'InstanceHardnessThreshold_xgb': Pipeline([('sampler', InstanceHardnessThreshold()),
    #                                 ('classifier', XGBClassifier())]),
    # 'InstanceHardnessThreshold_xgb_pt': Pipeline([('sampler', InstanceHardnessThreshold()),
    #                                 ('classifier', XGBClassifier(**xgb_best_params))])

}

In [ ]:
# for t, p in param_distribution_us.items():
#     print(t)
#     print(p)
#     param_grid = {f"sampler__{key}": value for key, value in param_distribution_us[technique].items()}
#     print(param_grid)

# for k, v in pipeline.items():
#   print(k)
#   print(v['sampler'])
#   print(v['classifier'])

In [2]:
#
best_params =  {}
best_score = {}

for technique, pipeline_obj in pipeline.items():
    print(f"Technique: {technique}") # key
    print(f"pipeline object: {pipeline_obj}") # value

    sampler_name = technique.split('_')[0] # because for example in pipeline RandomUnderSampler_rfc
    param_grid = {f"sampler__{key}": value for key, value in param_distribution_us[sampler_name].items()}
    # for example :

    print(f"Parameter grid: {param_grid}")

    # taking time to get the best parameters
    # rscv = RandomizedSearchCV(
    #     pipeline_obj,
    #     #param_distributions = param_distribution_us[technique],
    #     param_distributions = param_grid,
    #     n_iter=5,  # Number of parameter combinations to try
    #     cv=5,       # 5-fold cross-validation
    #     scoring='f1',  # Metric to optimize (F1 score for binary classification)
    #     verbose=2,   # Print progress
    #     random_state=42,
    #     n_jobs=-1    # Use all available cores
    # )
    # print(f"Shape of the data: {X_train.shape}\n")
    # X_resampled, y_resampled = pipeline_obj['sampler'].fit_resample(X_train, y_train)
    # print(f"Shape of the data, After resampling with {technique}: {X_resampled.shape}\n")
    # print(f"Fitting the data\n")

    # now trying BayesSearchCV

    opt = BayesSearchCV(
        pipeline_obj,
        param_grid,
        n_iter = 5,
        cv = 5,
        scoring = 'f1',
        verbose = 2,
        random_state = 42,
        n_jobs = -1
    )

    opt.fit(X_train, y_train)

    best_params[technique] = opt.best_params_
    best_score[technique] = opt.best_score_
    print(f"sampler: {pipeline_obj['sampler']} | classifier: {pipeline_obj['classifier']}")
    print(f"best parameters: {opt.best_params_}")
    print(f"best score: {opt.best_score_}")
    print('#'*30)
    print("\n")

## PARAMETER TUNING FOR OVERSAMPLING TECHNIQUE

In [ ]:
param_distribution_os = {
    'RandomOverSampler': {
        'sampling_strategy': ['majority', 'not minority', 'not majority', 'all', 'auto'],
        'random_state': [42],
        'shrinkage': [0.1, 0.3, 0.5]
    },
    'SMOTE': {
         'sampling_strategy': ['majority', 'not minority', 'not majority', 'all', 'auto'],
        'random_state': [42],
        'k_neighbors': [3, 5]
    },
    'SMOTENC': {
        'sampling_strategy': ['majority', 'not minority', 'not majority', 'all', 'auto'],
        'random_state': [42],
        'k_neighbors': [3, 5],
    },
    #  'SMOTEN': { #  for only categorical features
    #     'sampling_strategy': ['majority', 'not minority', 'not majority', 'all', 'auto'],
    #     'random_state': [42],
    #     'k_neighbors': [2, 3, 5],
    # },
    'ADASYN': {
        'sampling_strategy': ['majority', 'not minority', 'not majority', 'all', 'auto'],
         'random_state': [42],
        'n_neighbors': [3, 5]

    },
    'BorderlineSMOTE': {
        'sampling_strategy': ['majority', 'not minority', 'not majority', 'all', 'auto'],
        'random_state':[42],
        'k_neighbors': [3, 5],
        'm_neighbors': [3, 5],
        'kind': ['borderline-1', 'borderline-2']
    },
    'KMeansSMOTE': {
        'sampling_strategy': ['majority', 'not minority', 'not majority', 'all', 'auto'],
        'random_state':[42],
        'n_jobs': -1,
        'k_neighbors': [3, 5]
    },
    'SVMSMOTE': {
        'sampling_strategy': ['majority', 'not minority', 'not majority', 'all', 'auto'],
        'random_state':[42],
        'k_neighbors': [3, 5],
        'm_neighbors': [3, 5],
        # 'n_neighbors_ver3':[3, 5, 7],
        # 'n_jobs': -1
        'out_step': [0.3, 0.5, 0.7]
    }
}

pipeline = {
    'RandomOverSampler_rfc': Pipeline([('sampler', RandomOverSampler()),
                                    ('classifier', RandomForestClassifier(random_state=42))]),
    # 'RandomOverSampler_rfc_pt': Pipeline([('sampler', RandomUnderSampler()),
    #                                 ('classifier', RandomForestClassifier(**rfc_best_params,random_state=42))]),
    # 'RandomOverSampler_xgb': Pipeline([('sampler', RandomUnderSampler()),
    #                                 ('classifier', XGBClassifier())]),
    # 'RandomOverSampler_xgb_pt': Pipeline([('sampler', RandomUnderSampler()),
    #                                 ('classifier', XGBClassifier(**xgb_best_params))]),

    'SMOTE_rfc': Pipeline([('sampler', SMOTE()),
                                    ('classifier', RandomForestClassifier(random_state=42))]),
    # 'SMOTE_rfc_pt': Pipeline([('sampler', EditedNearestNeighbours()),
    #                                 ('classifier', RandomForestClassifier(**rfc_best_params,random_state=42))]),
    # 'SMOTE_xgb': Pipeline([('sampler', EditedNearestNeighbours()),
    #                                 ('classifier', XGBClassifier())]),
    # 'SMOTE_xgb_pt': Pipeline([('sampler', EditedNearestNeighbours()),
    #                                 ('classifier', XGBClassifier(**xgb_best_params))]),


    # 'SMOTENC_rfc': Pipeline([('sampler', CondensedNearestNeighbour()),
    #                                 ('classifier', RandomForestClassifier(random_state=42))]),

    # 'SMOTENC_rfc_pt': Pipeline([('sampler', CondensedNearestNeighbour()),
    #                                 ('classifier', RandomForestClassifier(**rfc_best_params,random_state=42))]),

    # 'SMOTENC_xgb': Pipeline([('sampler', CondensedNearestNeighbour()),
    #                                 ('classifier', XGBClassifier())]),

    # 'SMOTENC_xgb_pt': Pipeline([('sampler', CondensedNearestNeighbour()),
    #                                 ('classifier', XGBClassifier(**xgb_best_params))]),


    'SMOTENC_rfc': Pipeline([('sampler', SMOTENC()),
                                    ('classifier', RandomForestClassifier(random_state=42))]),
    # 'ADASYN_rfc_pt': Pipeline([('sampler', TomekLinks()),
    #                                 ('classifier', RandomForestClassifier(**rfc_best_params,random_state=42))]),
    # 'ADASYN_xgb': Pipeline([('sampler', TomekLinks()),
    #                                 ('classifier', XGBClassifier())]),
    # # 'ADASYN_xgb_pt': Pipeline([('sampler', TomekLinks()),
    #                                 ('classifier', XGBClassifier(**xgb_best_params))]),


    'ADASYN_rfc': Pipeline([('sampler', ADASYN()),
                                    ('classifier', RandomForestClassifier(random_state=42))]),
    # 'BorderlineSMOTE_rfc_pt': Pipeline([('sampler', ClusterCentroids()),
    #                                 ('classifier', RandomForestClassifier(**rfc_best_params,random_state=42))]),
    # 'BorderlineSMOTE_xgb': Pipeline([('sampler', ClusterCentroids()),
    #                                 ('classifier', XGBClassifier())]),
    # 'BorderlineSMOTE_xgb_pt': Pipeline([('sampler', ClusterCentroids()),
    #                                 ('classifier', XGBClassifier(**xgb_best_params))]),


    'KMeansSMOTE_rfc': Pipeline([('sampler', KMeansSMOTE()),
                                    ('classifier', RandomForestClassifier(random_state=42))]),
    # 'KMeansSMOTE_rfc_pt': Pipeline([('sampler', RepeatedEditedNearestNeighbours()),
    #                                 ('classifier', RandomForestClassifier(**rfc_best_params,random_state=42))]),
    # 'KMeansSMOTE_xgb': Pipeline([('sampler', RepeatedEditedNearestNeighbours()),
    #                                 ('classifier', XGBClassifier())]),
    # 'KMeansSMOTE_xgb_pt': Pipeline([('sampler', RepeatedEditedNearestNeighbours()),
    #                                 ('classifier', XGBClassifier(**xgb_best_params))]),


    'BorderlineSMOTE_rfc': Pipeline([('sampler', BorderlineSMOTE()),
                                    ('classifier', RandomForestClassifier(random_state=42))]),
    # 'NearMiss_rfc_pt': Pipeline([('sampler', NearMiss()),
    #                                 ('classifier', RandomForestClassifier(**rfc_best_params,random_state=42))]),
    # 'NearMiss_xgb': Pipeline([('sampler', NearMiss()),
    #                                 ('classifier', XGBClassifier())]),
    # 'NearMiss_xgb_pt': Pipeline([('sampler', NearMiss()),
    #                                 ('classifier', XGBClassifier(**xgb_best_params))]),


    # 'SMOTEN_rfc': Pipeline([('sampler', SMOTEN()),
    #                                 ('classifier', RandomForestClassifier(random_state=42))]),


    # 'NeighbourhoodCleaningRule_rfc_pt': Pipeline([('sampler', NeighbourhoodCleaningRule()),
    #                                 ('classifier', RandomForestClassifier(**rfc_best_params,random_state=42))]),

    # 'NeighbourhoodCleaningRule_xgb': Pipeline([('sampler', NeighbourhoodCleaningRule()),
    #                                 ('classifier', XGBClassifier())]),

    # 'NeighbourhoodCleaningRule_xgb_pt': Pipeline([('sampler', NeighbourhoodCleaningRule()),
    #                                 ('classifier', XGBClassifier(**xgb_best_params))]),

    'SVMSMOTE_rfc': Pipeline([('sampler', SVMSMOTE()),
                                    ('classifier', RandomForestClassifier(random_state=42))]),
    # 'SVMSMOTE_rfc_pt': Pipeline([('sampler', OneSidedSelection()),
    #                                 ('classifier', RandomForestClassifier(**rfc_best_params,random_state=42))]),
    # 'SVMSMOTE_xgb': Pipeline([('sampler', OneSidedSelection()),
    #                                 ('classifier', XGBClassifier())]),
    # 'SVMSMOTE_xgb_pt': Pipeline([('sampler', OneSidedSelection()),
    #                                 ('classifier', XGBClassifier(**xgb_best_params))])

}

### PARAMETER TUNING FOR OVER AND UNDER SAMPLING TECHNIQUE

In [ ]:
param_distribution_mixed = {
    'SMOTEENN': {
        'sampling_strategy': ['majority', 'not minority', 'not majority', 'all', 'auto'],
        'random_state': [42],
        'n_jobs': -1,

    },
    'SMOTETomek': {
         'sampling_strategy': ['majority', 'not minority', 'not majority', 'all', 'auto'],
        'random_state': [42],
        'k_neighbors': [2, 3, 5]
    }
}

pipeline = {
    'SMOTEENN_rfc': Pipeline([('sampler', SMOTEENN()),
                                    ('classifier', RandomForestClassifier(random_state=42))]),
    # 'SMOTEENN_rfc_pt': Pipeline([('sampler', SMOTEENN()),
    #                                 ('classifier', RandomForestClassifier(**rfc_best_params,random_state=42))]),
    # 'SMOTEENN_xgb': Pipeline([('sampler', SMOTEENN()),
    #                                 ('classifier', XGBClassifier())]),
    # 'SMOTEENN_xgb_pt': Pipeline([('sampler', SMOTEENN()),
    #                                 ('classifier', XGBClassifier(**xgb_best_params))]),

    'SMOTETomek_rfc': Pipeline([('sampler', SMOTETomek()),
                                    ('classifier', RandomForestClassifier(random_state=42))]),
    # 'SMOTETomek_rfc_pt': Pipeline([('sampler', SMOTETomek()),
    #                                 ('classifier', RandomForestClassifier(**rfc_best_params,random_state=42))]),
    # 'SMOTETomek_xgb': Pipeline([('sampler', SMOTETomek()),
    #                                 ('classifier', XGBClassifier())]),
    # 'SMOTETomek_xgb_pt': Pipeline([('sampler', SMOTETomek()),
    #                                 ('classifier', XGBClassifier(**xgb_best_params))])
}